In [1]:
import bashlex
import copy
import re
import os

from tqdm import tqdm

In [2]:
re_command_filter = "C"
re_error_filter = "X"
re_start_filter = "S" 

In [3]:
scientists_dir = "./unix-data/computer-scientists/"
experienced_dir = "./unix-data/experienced-programmers/"
non_programmers_dir = "./unix-data/non-programmers/"
novice_dir = "./unix-data/novice-programmers/"

scientists_files = os.listdir(scientists_dir)
experienced_files = os.listdir(experienced_dir)
non_programmers_files = os.listdir(non_programmers_dir)
novice_files = os.listdir(novice_dir)

scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_parsed = []
experienced_parsed = []
non_programmers_parsed = []
novice_parsed = []

Generates list of files

In [4]:
for i in range(len(scientists_files)):
    scientists_files[i] = scientists_dir + scientists_files[i]

for i in range(len(experienced_files)):
    experienced_files[i] = experienced_dir + experienced_files[i]

for i in range(len(non_programmers_files)):
    non_programmers_files[i] = non_programmers_dir + non_programmers_files[i]

for i in range(len(novice_files)):
    novice_files[i] = novice_dir + novice_files[i]

Parse function that returns nested list of parsed commands based on session

In [15]:
def parse_commands_per_session(command_list, parsed_list, files_list):
    for file_path in tqdm(files_list):
        file1 = open(file_path, encoding="ISO-8859-1")
        lines = file1.readlines()

        command_sublist = []
        parsed_sublist = []

        for line in range(len(lines)):
            
            if re.match(re_start_filter, lines[line]) is not None:
                if len(command_sublist) != 0 and len(parsed_sublist) != 0:
                    command_list.append(copy.deepcopy(command_sublist))
                    parsed_list.append(copy.deepcopy(parsed_sublist))


                command_sublist = []
                parsed_sublist = []
            
            if re.match(re_command_filter, lines[line]) is not None:
                command_sublist.append(lines[line][2:-1])
                
                try:
                    parts = list(bashlex.split(command_sublist[-1]))
                    parsed_sublist.append(parts)
                except Exception as inst:
                    command_sublist.pop(-1)
                    
    
    # Not necessary as list is created by ref parameter 
    return command_list
            

Function that parses just by command

In [5]:
def parse_commands(command_list, parsed_list, files_list):
    for file_path in tqdm(files_list): 
        file1 = open(file_path, encoding="ISO-8859-1")
        lines = file1.readlines()

        for line in range(len(lines)):
            if re.match(re_command_filter, lines[line]) is not None:
                command_list.append(lines[line][2:-1])
                
                try:
                    parts = bashlex.parse(command_list[-1])
                    parsed_list.append(parts)
                except Exception as inst:
                    command_list.pop(-1)

    return command_list

Function that parses commands and groups into subsets of `subset_size` meaning command chains are `subset_size` commands long.

In [6]:
def parse_commands_into_subsets(command_list, parsed_list, files_list, subset_size):
    command_list = copy.deepcopy(parse_commands(command_list, parsed_list, files_list))

    filter_escaped = lambda x: (x and x.isprintable())
    command_list = list(filter(filter_escaped, command_list))

    result = []

    for i in range(0, len(command_list), subset_size):
        result.append(command_list[i:i+subset_size])

    command_list = result

    return command_list


In [18]:
scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_commands = parse_commands_per_session(scientist_commands, scientist_parsed, scientists_files)
experienced_commands = parse_commands_per_session(experienced_commands, experienced_parsed, experienced_files)
non_programmers_commands = parse_commands_per_session(non_programmers_commands, non_programmers_parsed, non_programmers_files)
novice_commands = parse_commands_per_session(novice_commands, novice_parsed, novice_files)

filter_empty = lambda x: (len(x) > 0)

scientist_commands = list(filter(filter_empty, scientist_commands))
experienced_commands = list(filter(filter_empty, experienced_commands))
non_programmers_commands = list(filter(filter_empty, non_programmers_commands))
novice_commands = list(filter(filter_empty, novice_commands))

100%|██████████| 56/56 [00:04<00:00, 13.33it/s]


In [19]:
subset_size = 5

scientist_commands = []
experienced_commands = []
non_programmers_commands = []
novice_commands = []

scientist_commands_subsets = parse_commands_into_subsets(scientist_commands, scientist_parsed, scientists_files, subset_size)
experienced_commands_subsets = parse_commands_into_subsets(experienced_commands, experienced_parsed, experienced_files, subset_size)
non_programmers_commands_subsets = parse_commands_into_subsets(non_programmers_commands, non_programmers_parsed, non_programmers_files, subset_size)
novice_commands_subsets = parse_commands_into_subsets(novice_commands, novice_parsed, novice_files, subset_size)

filter_empty = lambda x: (x is not None)

scientist_commands_subsets = list(filter(filter_empty, scientist_commands_subsets))
experienced_commands_subsets = list(filter(filter_empty, experienced_commands_subsets))
non_programmers_commands_subsets = list(filter(filter_empty, non_programmers_commands_subsets))
novice_commands_subsets = list(filter(filter_empty, novice_commands_subsets))

100%|██████████| 56/56 [00:07<00:00,  7.45it/s]


In [5]:
print(len(scientist_commands_subsets))
print(len(experienced_commands_subsets))
print(len(non_programmers_commands_subsets))
print(len(novice_commands_subsets))

print(scientist_commands_subsets[1])

NameError: name 'scientist_commands_subsets' is not defined

Defunct, predicts next argument in command with this structure

In [1]:
from graph import Node

def construct_graph_defunct(parsed_commands, command_dict = {}):

    for cmd in parsed_commands:

        cur_node = None

        if command_dict.get(cmd[0]) is None:
            cur_node = Node(command = cmd[0], frequency = 1)
            command_dict[cmd[0]] = cur_node
        else:
            command_dict.get(cmd[0]).frequency += 1
            cur_node = command_dict.get(cmd[0])

        for i in range(1, len(cmd)):
            child_node = None

            if cur_node.children.get(cmd[i]) is None:
                child_node = Node(cmd[i], 1)
                cur_node.children[cmd[i]] = child_node
            else:
                cur_node.children.get(cmd[i]).frequency += 1
                child_node = cur_node.children.get(cmd[i])
            
            cur_node = child_node
        
    return command_dict


Graph structure for next command. Use command list instead of bashlex output since we are comparing whole commands as nodes in a graph.

Making command dict key the program (e.g. `cat`) and value the node with that program and have that nodes children be the full command. And then have the children of those nodes be determined by fuzzy matching. So a node can be a child to many parents if its fuzzy matched.

In [7]:
from graph import Node

def construct_graph(command_list, command_dict={}):

    filter_empty = lambda x: (len(x) > 0)
    cur_node = None
    child_node = None
    
    for session in command_list:
        # remove commands of length 0 ( not good to modify list within loop )
        
        session = list(filter(filter_empty, session))

        try: 
            first_cmd = session[0]
        except Exception as inst:
            print(session)
            continue

        program = session[0].split()[0]

        if command_dict.get(program) is None:
            cur_node = Node(program=program, frequency=1)
            command_dict[program] = cur_node 
        else:
            cur_node = command_dict.get(program)
            cur_node.frequency += 1

        if cur_node.commands.get(first_cmd) is None:
            cur_node.commands[first_cmd] = 1
        else:
            cur_node.commands[first_cmd] += 1

        for cmd in range(1, len(session)):

            if session[cmd].isprintable() is False:
                break
            else:
                program = session[cmd].split()[0]

            if cur_node.children.get(program) is None:
                child_node = Node(program = program, frequency=1)
                cur_node.children[program] = child_node
            else:
                child_node = cur_node.children.get(program)
                child_node.frequency += 1
            
            if child_node.commands.get(session[cmd]) is None:
                child_node.commands[session[cmd]] = 1
            else:
                child_node.commands[session[cmd]] += 1
            
            cur_node = child_node
        
                
        
    return command_dict


Takes list of commands and observes last 4 to predict 5th

In [8]:
def get_prediction(command_list, graph):
    if len(command_list) == 0:
        return None

    commands = command_list[-3:-1]
    previous_command = command_list[-1]

    if len(commands) == 0:
        return None

    program = commands[0].split()[0]
    
    if graph.get(program) is None:
        return None
    else:
        node = graph[program]

    for command in commands[1:]:
        program = command.split()[0]
        if node.children.get(program) is not None:
            node = node.children[program]
        else:
            return None

    return node.get_prediction(previous_command)


In [14]:
from thefuzz import fuzz
from thefuzz import process
from sklearn.model_selection import train_test_split

import pprint
pp = pprint.PrettyPrinter(indent=4)

def get_accuracy(command_subsets, print_fails = False):
    split_ratio = .8
    train_data, test_data = train_test_split(command_subsets, test_size=0.2, train_size=0.8, random_state=42)

    train_graph = construct_graph(train_data)

    test_size = len(test_data)
    correct = 0

    for commands in test_data:
        results = get_prediction(commands, train_graph)
        
        prev_correct = correct

        if results is not None:
            for result in results:
                if fuzz.ratio(result[0], commands[-1]) > 75:
                    correct += 1
                    break
        if prev_correct == correct and print_fails:
            pp.pprint(("results:", results, "expected:" , commands[-1], "command sequence:", commands))
                    

    return '{:.2f}%'.format(100 * correct/test_size)


Cyverse command pull and parse

In [ ]:
import requests
import time

url = "https://us-south.functions.appdomain.cloud/api/v1/web/ORG-UNC-dist-seed-james_dev/cyverse/get-cyverse-log"
query = {'body': {'log_type':'Bash', 'password': 'password', 'skip': '0', 'limit':'100'}}

response_dict = {}

def get_cyverse_commands():
    for i in range(0, 1000, 100):
        query = {'body': {'log_type':'Bash', 'password': 'password', 'skip': str(i), 'limit':'100'}}
        response = requests.get(url, headers = {"Content-Type": "application/json"}, json = query)
        print(i, response.status_code)
        response_dict.update(response.json())
        time.sleep(50)

    return response_dict

get_cyverse_commands()


In [ ]:
cyverse_commands = []

for log in response_dict['logs']:
    for command in log['log']['commands']:
        if 'full_command' in command:
            cyverse_commands.append(command['full_command'])

len(cyverse_commands)

In [ ]:
def parse_cyverse_subsets(command_list, subset_size):
    result = []

    for i in range(0, len(command_list), subset_size):
        result.append(command_list[i:i+subset_size])

    return result

cyverse_commands_subsets = parse_cyverse_subsets(cyverse_commands, 5)

get_accuracy(cyverse_commands_subsets)

Colearning attempt with `scientist-1` and `scientist-2`

In [43]:
def replace_args(parsed_list):
        filter_escaped = lambda x: (x and x.isprintable())

        parsed_list_replaced = []
        arg_replacements = ["$" + str(i) for i in range(100)]

        exclude_chars = set(['|', '<', '>'])
        
        for lst in parsed_list:
            lst = list(filter(filter_escaped, lst))
            parsed_sublist_replaced = []
            for command in range(len(lst)):
                command_split = lst[command].split()
                arg_counter = 0

                for i in range(1, len(command_split)):
                    if not command_split[i].startswith("-") and command_split[i] not in exclude_chars:
                        command_split[i] = arg_replacements[arg_counter]
                        arg_counter += 1
                
                parsed_sublist_replaced.append(' '.join(command_split))
                
            parsed_list_replaced.append(parsed_sublist_replaced)

        return parsed_list_replaced

In [50]:
def get_accuracy_colearn(command_subsets, test_set, print_fails = False):

    train_graph = construct_graph(command_subsets)

    test_size = len(test_set)
    correct = 0

    for commands in test_set:
        results = get_prediction(commands, train_graph)
        
        prev_correct = correct

        if results is not None:
            for result in results:
                if fuzz.partial_ratio(result[0], commands[-1]) > 85:
                    correct += 1
                    break
        if prev_correct == correct and print_fails:
            pp.pprint(("results:", results, "expected:" , commands[-1], "command sequence:", commands))
                    

    return '{:.2f}%'.format(100 * correct/test_size)


In [45]:
scientist_commands1 = parse_commands_into_subsets([], [], scientists_files[0:1], 5)
scientist_commands1 = replace_args(scientist_commands1)

scientist_commands2 = parse_commands_into_subsets([], [], scientists_files[1:2], 5)
scientist_commands2 = replace_args(scientist_commands2)

scientist_commands3 = parse_commands_into_subsets([], [], scientists_files[2:3], 5)
scientist_commands3 = replace_args(scientist_commands3)

scientist_commands10 = parse_commands_into_subsets([], [], scientists_files[0:10], 5)
scientist_commands10 = replace_args(scientist_commands10)

100%|██████████| 10/10 [00:03<00:00,  2.72it/s]


In [51]:
print(get_accuracy_colearn(scientist_commands1, scientist_commands2, print_fails=False))

print(get_accuracy_colearn(scientist_commands2, scientist_commands1, print_fails=False))

print(get_accuracy_colearn(scientist_commands1, scientist_commands3, print_fails=False))

print(get_accuracy_colearn(scientist_commands3, scientist_commands1, print_fails=False))

print(get_accuracy_colearn(scientist_commands2, scientist_commands3, print_fails=False))

print(get_accuracy_colearn(scientist_commands3, scientist_commands2, print_fails=False))

print(get_accuracy_colearn(scientist_commands10, scientist_commands2, print_fails=False))

print(get_accuracy_colearn(scientist_commands2, scientist_commands10, print_fails=False))

96.72%
94.57%
95.79%
94.57%
95.79%
96.72%
96.72%
96.75%


In [52]:
science_subsets_graph = parser.replace_args(parser.scientists_commands)
science_subsets_graph = parser.parse_commands_into_subsets(science_subsets_graph, 5)
graph = construct_graph(science_subsets_graph)

In [56]:
graph["cc"].children["mv"].commands

{'mv $0 $1': 32,
 'mv wancluster.c wancluster.c.old': 1,
 'mv a.out dograms': 1,
 'mv timeofda.h timeofday.h': 1,
 'mv marbleli.h marblelist.h': 1,
 'mv marblecl.h marbleclock.h': 1,
 'mv a.out getim': 1,
 'mv a.out header': 1,
 'mv a.out subim': 1,
 'mv a.out ../bin/shell': 1}